### Quiz Builder

In [6]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from IPython.display import Markdown

In [7]:
load_dotenv()

True

In [8]:
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai", )
response = model.invoke("Hello World!!")
print(response)

content='Hello there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--07e79a4e-a80c-46d9-a755-4398223da6cb-0' usage_metadata={'input_tokens': 3, 'output_tokens': 11, 'total_tokens': 14, 'input_token_details': {'cache_read': 0}}


In [9]:
# Student profile type definition
class StudentProfile(TypedDict):
    gender: str
    class_level: str  # e.g., "class 6", "12th", "undergrad", "postgrad"
    language: str     # e.g., "hindi", "english", "marathi"


class UserPrompt(TypedDict):
    topic: str
    pdf_file: str
    audio_file: str

# Graph state


class State(TypedDict):
    student_profile: StudentProfile
    user_prompt: UserPrompt
    summary_notes: str
    podcast_script: str
    mindmap: str
    quiz: str
    recommended_resources: str
    study_plan: str
    combined_output: str

In [10]:
# Prompts Templates
class PersonalizedPromptsStudent:
    def __init__(self, topic, student_profile: dict, user_instructions: str = ""):
        self.topic = topic
        self.student_profile = student_profile
        self.user_instructions = user_instructions

    def summary_notes(self):
        """Create a personalized prompt based on student profile"""

        template = ChatPromptTemplate([
            ("system", """You are an expert academic tutor. Create personalized educational content following these guidelines:
            
            Student Profile:
            - Class Level: {class_level}
            - Language: {language} 
            - Gender: {gender}
            
            Content Requirements:
            1. Summary notes in maximum {word_limit} words
            2. Use bullet points and simple language appropriate for {class_level}
            3. Include practical examples and analogies
            4. Make it engaging and easy to understand
            5. If language is not English, provide content in {language}
            
            Additional Instructions: {user_instructions}"""),

            ("user", "Please teach me about: {topic}")
        ])

        return template.invoke({
            "topic": self.topic,
            "class_level": self.student_profile.get("class_level", "general"),
            "language": self.student_profile.get("language", "English"),
            "gender": self.student_profile.get("gender", ""),
            "word_limit": "150",
            "user_instructions": self.user_instructions or "Focus on key concepts with examples"
        })

In [11]:
def node_summary_notes(state: State):
    """LLM call to generate summary notes for student"""

    summary_prompt_template = PersonalizedPromptsStudent(
        state['user_prompt']['topic'], state['student_profile'])

    summary_prompt = summary_prompt_template.summary_notes()

    msg = model.invoke(summary_prompt)

    return {"summary_notes": msg.content}

In [12]:
input_state = {
    "student_profile": {
        "gender": "MALE",
        "class_level": "12th",
        "language": "HINDI"
    },
    "user_prompt": {
        "topic": "Chola kingdom"
    }
}

In [16]:
output = node_summary_notes(input_state)

In [17]:
Markdown(output["summary_notes"])

नमस्ते! चलो चोल साम्राज्य के बारे में सीखते हैं, जो दक्षिण भारत का एक शक्तिशाली और महत्वपूर्ण साम्राज्य था।

**चोल साम्राज्य (Chola Empire):**

*   **कौन थे?** चोल दक्षिण भारत के एक शक्तिशाली राजवंश थे, जिन्होंने लगभग 9वीं शताब्दी से 13वीं शताब्दी तक शासन किया।
*   **कहाँ थे?** उनका मुख्य क्षेत्र तमिलनाडु था, लेकिन उन्होंने दक्षिण भारत और श्रीलंका के कुछ हिस्सों पर भी नियंत्रण किया।
*   **क्या किया?**
    *   **मजबूत नौसेना:** चोलों के पास एक शक्तिशाली नौसेना थी, जिसने उन्हें समुद्री व्यापार और विजय में मदद की।
    *   **कला और वास्तुकला:** उन्होंने शानदार मंदिर बनवाए, जैसे कि तंजौर का बृहदेश्वर मंदिर। यह मंदिर उनकी कला और वास्तुकला का अद्भुत उदाहरण है।
    *   **प्रशासन:** उनकी प्रशासनिक व्यवस्था बहुत अच्छी थी। उन्होंने स्थानीय स्वशासन को बढ़ावा दिया, जिससे गाँवों को अपने फैसले लेने की शक्ति मिली।
*   **उदाहरण:** सोचो, चोल साम्राज्य एक बड़ी कंपनी की तरह था, जिसके पास अलग-अलग विभाग थे (जैसे नौसेना, कला, प्रशासन), और हर विभाग अपने काम में माहिर था!
*   **खास बातें:**
    *   चोलों ने व्यापार को बढ़ावा दिया, जिससे उनका साम्राज्य समृद्ध हुआ।
    *   उन्होंने शिक्षा और साहित्य को भी प्रोत्साहित किया।

यह चोल साम्राज्य का एक संक्षिप्त परिचय है। यह साम्राज्य अपनी शक्ति, कला, और संस्कृति के लिए इतिहास में हमेशा याद किया जाएगा।